# Turkish XTTS Fine-tuning Setup

**IMPORTANT:** Restart runtime first (`Runtime` → `Restart runtime`), then run these cells in order.

In [ ]:
# Step 1: Install compatible packages (run FIRST after restart)
!pip install -q "numpy==1.26.4" "scipy==1.11.4"
!pip install -q torch==2.3.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q "TTS==0.22.0" "trainer==0.0.36"

In [ ]:
# Step 2: Verify imports
import sys, numpy, scipy, torch
print(sys.version)
print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())

from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
print("TTS import OK")

In [ ]:
# Step 3: Clone repository and mount Google Drive
%cd /content
!git clone https://github.com/ElkhanAbbasov/aiswhis
%cd aiswhis

from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/xtts_tr_dataset"

In [ ]:
# Step 4: Patch training script to disable eval
%cd /content/aiswhis
from pathlib import Path
import re

p = Path("train_gpt_xtts_tr.py")
s = p.read_text()

s = s.replace("eval_split=True", "eval_split=False")

s = re.sub(
    r"(train_samples\s*,\s*eval_samples\s*=\s*load_tts_samples\([^)]*\))",
    r"\1\n    eval_samples = None",
    s,
    count=1,
)

s = s.replace("run_eval=True", "run_eval=False")

p.write_text(s)
print("train_gpt_xtts_tr.py patched.")

In [ ]:
# Step 5: Run training
%cd /content/aiswhis
!python train_gpt_xtts_tr.py